# Building and Deploying a Machine Learning Model with FastAPI

---

In this tutorial, we'll be building a simple API using Python libraries like Scikit-learn (for machine learning), Joblib (for saving the model), FastAPI (for building the API), and Uvicorn (to run the API).  We'll use a classic dataset  - breast cancer diagnosis - to train a model and then see how to use our API to make predictions on new data.

## Step 1: Environment Setup
First, ensure you have Python installed on your system. Then, install the necessary libraries:

In [1]:
pip install numpy pandas scikit-learn fastapi uvicorn joblib

Note: you may need to restart the kernel to use updated packages.


## Step 2: Prepare and Train the Model
We'll use the Breast Cancer dataset provided by Scikit-learn and train a simple logistic regression model.

In [3]:
%%writefile ml.py
# This code will be written to the 'ml.py' file

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from joblib import dump

# Load the dataset
data = load_breast_cancer()
X = data.data
y = data.target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the model
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

# Save the model and scaler
dump(model, 'breast_cancer_model.joblib')
dump(scaler, 'scaler.joblib')
print("Successfuly saved the model and scaler joblib files..")

Overwriting ml.py


Run the following cell, and you should see `Successfuly saved the model and scaler joblib files.` message printed below the cell, indicating you've successfully executed all your machine learning codes in the `ml.py` file.

In [12]:
%run ml.py

Successfuly saved the model and scaler joblib files.


## Step 3: Setting Up FastAPI
Create a new Python file for the API server. We'll use FastAPI to set up endpoints that can receive new data and make predictions.

In [44]:
%%writefile main.py
from fastapi import FastAPI, HTTPException  # Import necessary components from FastAPI and HTTPException for error handling.
from pydantic import BaseModel  # Import BaseModel from Pydantic to define data models.
from typing import List  # Import List from typing to specify data types in data models.
from joblib import load  # Import load function from joblib to load pre-trained models.
import datetime

app = FastAPI()  # Create an instance of FastAPI to define and manage your web application.

# Load pre-trained model and scaler objects from disk. These are used for making predictions and scaling input data, respectively.
model = load('breast_cancer_model.joblib')
scaler = load('scaler.joblib')

# Define a data model for incoming prediction requests using Pydantic.
# This model ensures that data received via the API matches the expected format.
class QueryData(BaseModel):
    features: List[float]  # Define a list of floating point numbers to represent input features for prediction.


@app.get("/")
async def read_root():
    # Get the current date and time
    now = datetime.datetime.now()

    # Format the date and time according to your preference
    formatted_datetime = now.strftime("%d-%m-%Y %H:%M:%S")  # Example format (YYYY-MM-DD HH:MM:SS)

    # Print the formatted date and time
    return f"Hello, it is now {formatted_datetime}"

# Decorator to create a new route that accepts POST requests at the "/predict/" URL.
# This endpoint will be used to receive input data and return model predictions.
# Declaring async before a function definition is a way to handle asynchronous operations in FastAPI. 
# It allows the server to handle many requests efficiently by not blocking the server during operations 
# like network calls or while waiting for file I/O.
@app.post("/predict/")
async def make_prediction(query: QueryData):
    try:
        # The input data is received as a list of floats, which needs to be scaled (normalized) using the previously loaded scaler.
        scaled_features = scaler.transform([query.features])
        
        # Use the scaled features to make a prediction using the loaded model.
        # The model returns a list of predictions, and we take the first item since we expect only one.
        prediction = model.predict(scaled_features)
        
        # Return the prediction as a JSON object. This makes it easy to handle the response programmatically on the client side.
        return {"prediction": int(prediction[0])}
    except Exception as e:
        # If an error occurs during the prediction process, raise an HTTPException which will be sent back to the client.
        raise HTTPException(status_code=400, detail=str(e))


Overwriting main.py


#### REST (REpresentational State Transfer) in APIs

Imagine a restaurant (your API) serving data (the food). REST provides a set of guidelines for how to order and receive that data using standard HTTP methods (like placing an order):

* **GET:** Retrieve data
* **POST:** Create new data
* **PUT:** Update existing data
* **DELETE:** Remove data

#### RESTful APIs

A RESTful API adheres to these REST principles, making it a well-structured and widely adopted approach for building web APIs. It ensures your API is:

* **Client-Server:** Separates data providers (servers) from data consumers (clients). Data engineers design and maintain the server side, while other applications or tools can easily interact with it using the API.
* **Stateless:** Each request-response pair is independent, meaning the server doesn't need to remember past interactions for each request. This simplifies data engineering on the server side.
* **Standard Interface:** Uses HTTP verbs (GET, POST, PUT, DELETE) and common data formats (like JSON) for clear communication. This standardization makes it easier for other applications (and data engineers!) to understand and use your API.


**Real-World Example (Data Engineering):**

Suppose you have a data pipeline that pulls data from a database (server). It can use a RESTful API to:
- **GET:** Retrieve specific data sets using well-defined endpoints (URLs) with parameters.
- **POST:** Send new data entries to the database.
- **PUT:** Update existing records in the database.

#### FastAPI
FastAPI is a modern, fast (high-performance) web framework for building APIs with Python 3.7 or newer that are based on standard Python type hints. Type hints in Python allow developers to indicate the expected data types of function arguments and return values, improving code readability and facilitating error checking with tools.

Its key features include:
* **Automatic API documentation**: FastAPI automatically generates documentation (using Swagger UI and ReDoc) for your API from your Python code. This feature can be incredibly useful for both development and consumption of your API. No additional configuration is required to enable basic Swagger documentation in FastAPI by default. Access it at http://127.0.0.1:8000/docs after running the application. Swagger is an interactive API documentation tool that lets you explore and test APIs visually.
* **Type checking and validation**: By leveraging Python type hints, FastAPI provides automatic request validation and type conversions, reducing the amount of boilerplate code for parsing and validating input data.
* **Asynchronous support**: FastAPI is designed to be asynchronous from the ground up, which can lead to better performance under load, especially for I/O bound applications. This is a more advanced feature that may not be initially of concern for beginners but is valuable for building scalable applications.

## Step 4: Run the FastAPI API Server

* The `--reload` flag makes the server restart after code changes. **Not for production use**, but very useful for development.
* Visit http://127.0.0.1:8000 in your web browser. You should see the message `"Hello, it is now 2024-04-05 15:10:41"`.

* Launch the Terminal in VS Code. After that, run your app using `Uvicorn` by typing the following command at the prompt in the Terminal window:
```bash
        uvicorn main:app --reload
```

* `main`: This is the name of the Python module (a .py file without the extension) where your FastAPI application is defined. For example, if your FastAPI application is written inside a file named `main.py`, you would use `main` here.
* `app`: This is the variable name of the FastAPI application instance within that module. In a FastAPI application, you typically create the app with a line like `app = FastAPI()`. The `app` after the colon refers to this instance.

#### Uvicorn & FastAPI
Uvicorn is an ASGI (Asynchronous Server Gateway Interface) server implementation, written in Python. ASGI is a standard interface between web servers and web applications or frameworks for Python. It is designed to provide a standard way to build asynchronous applications in Python and to connect them with web servers.

Uvicorn serves as the link between the web and FastAPI, enabling the framework to operate on the web by handling incoming HTTP requests and sending responses. Its asynchronous nature boosts FastAPI's performance, making the duo a powerful toolkit for developing modern web applications and APIs.

* Uvicorn allows FastAPI to run asynchronously, handling multiple requests concurrently. 
* Uvicorn is lightning fast. It is designed to be efficient and to take advantage of modern CPUs. This complements FastAPI's performance, making the combination of the two an excellent choice for applications requiring high throughput.

Configuring Uvicorn for production differs significantly from setting it up for development. In development, convenience features like auto-reload are prioritized, while in production, stability, security, and performance take precedence. 
* In development, you often run Uvicorn with the --reload flag to automatically restart the server upon changes to the code, enhancing productivity. 
* For production systems, the automatic reload feature provided by Uvicorn with the `--reload` flag should **not** be used. 
  * The auto-reload feature is designed for development purposes to increase productivity by automatically restarting the server when code changes are detected. However, in a production environment, this feature could lead to instability and unnecessary downtime, as any change to the codebase would restart the server, potentially interrupting active connections and requests. Auto-reloading introduces overhead because the server needs to monitor file changes constantly. In a production environment, the additional overhead from watching for file changes can detract from the application's performance.
  * To configure Uvicorn for production, you should **remove** the `--reload` flag and follow best practices for stability and performance. If you're using Uvicorn directly, simply **omit** the `--reload` option when starting the server: `uvicorn main:app`


## Step 5: Testing the API Using Python
First, ensure you have the requests library installed:

In [ ]:
pip install requests

Now, create a Python script (test_api.py) to send POST requests with dummy data to your API:

In [19]:
%%writefile test_api.py
import requests
import json

# Define the API endpoint
url = "http://127.0.0.1:8000/predict/"

# Sample data with 30 dummy feature values
data = {
    "features": [
        1.799e+01, 1.038e+01, 1.228e+02, 1.001e+03, 1.184e-01, 2.776e-01, 3.001e-01, 
        1.471e-01, 2.419e-01, 7.871e-02, 1.095e+00, 9.053e-01, 8.589e+00, 1.534e+02, 
        6.399e-03, 4.904e-02, 5.373e-02, 1.587e-02, 3.003e-02, 6.193e-03, 2.538e+01, 
        1.733e+01, 1.846e+02, 2.019e+03, 1.622e-01, 6.656e-01, 7.119e-01, 2.654e-01, 
        4.601e-01, 1.189e-01
    ]
}

# Send a POST request
response = requests.post(url, json=data)

# Print the response
print("Status Code:", response.status_code)
print("Response Body:", response.json())


Overwriting test_api.py


In [45]:
%run test_api.py

Status Code: 200
Response Body: {'prediction': 0}


In [46]:
%%writefile dummy.csv
1.799e+01,1.038e+01,1.228e+02,1.001e+03,1.184e-01,2.776e-01,3.001e-01,1.471e-01,2.419e-01,7.871e-02,1.095e+00,9.053e-01,8.589e+00,1.534e+02,6.399e-03,4.904e-02,5.373e-02,1.587e-02,3.003e-02,6.193e-03,2.538e+01,1.733e+01,1.846e+02,2.019e+03,1.622e-01,6.656e-01,7.119e-01,2.654e-01,4.601e-01,1.189e-01

Overwriting dummy.csv


In [47]:
%%writefile test_api_csv.py
import requests
import json
import pandas as pd

# Define the API endpoint
url = "http://127.0.0.1:8000/predict/"

# Read data from dummy.csv using pandas
try:
  data = pd.read_csv("dummy.csv", header=None).values.tolist()
except FileNotFoundError:
  print("Error: File 'dummy.csv' not found. Please ensure the file exists.")
  exit(1)

# Ensure data contains 30 features
if len(data[0]) != 30:
  print("Error: 'dummy.csv' does not contain 30 features. Please check the file format.")
  exit(1)

# Convert data to a list of floats (assuming each row has 30 features)
data = [float(value) for value in data[0]]  # Access the first row

# Prepare data dictionary
data = {
  "features": data
}

# Send a POST request
response = requests.post(url, json=data)

# Print the response
print("Status Code:", response.status_code)
print("Response Body:", response.json())


Overwriting test_api_csv.py


In [48]:
%run test_api_csv.py

Status Code: 200
Response Body: {'prediction': 0}


### Expected Output:
* **Status Code**: 
  * 200 indicates that the HTTP request was successfully received, understood, and processed by the server.
* **Response Body**: 
  * This part will show the prediction result returned by the machine learning model. 
  * In this case, 0 indicates "no cancer detected" (depending on how the labels are encoded).

## Step 6: Stop the Uvicorn server
If you no longer need the Uvicorn server, press `CTRL+C` in the Terminal window to stop the server. For macOS, press `CMD+C` in the Terminal window. 
